<center><h1>Projet : Partie 5</h1></center>

<center><h1>Algorithmes de Clustering</h1></center>

## Importation des librairies et du dataset 

In [145]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import warnings
from collections import Counter
warnings.filterwarnings("ignore")

In [146]:
df = pd.read_excel("data.xlsx")
df.drop(columns = df.columns[0], axis = 1, inplace= True)
df.drop(columns = df.columns[1], axis = 1, inplace= True)
df.head()

,Age,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,Gender,HourlyRate,...,PercentSalaryHike,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,0.56,2,0.72,Sales,0.00,0.25,Life Sciences,0.33,0,0.93,...,0.00,0.00,0.0,0.40,0.00,0.00,0.16,0.24,0.00,0.38
1,0.76,1,0.13,Research & Development,0.26,0.00,Life Sciences,0.67,1,0.45,...,0.86,1.00,0.5,0.50,0.50,0.67,0.27,0.41,0.07,0.54
2,0.46,2,0.91,Research & Development,0.04,0.25,Other,1.00,1,0.90,...,0.29,0.33,0.0,0.35,0.50,0.67,0.00,0.00,0.00,0.00
3,0.37,1,0.92,Research & Development,0.07,0.75,Life Sciences,1.00,0,0.38,...,0.00,0.67,0.0,0.40,0.50,0.67,0.22,0.41,0.21,0.00
4,0.34,1,0.65,Research & Development,0.04,0.25,Life Sciences,1.00,1,0.71,...,0.14,0.67,0.0,0.40,0.33,0.33,0.19,0.41,0.21,0.46


In [147]:
data = df.select_dtypes(include='number') #on ne garde que les variables numériques
data.head()

,Age,BusinessTravel,DailyRate,DistanceFromHome,Education,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,...,PercentSalaryHike,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,0.56,2,0.72,0.00,0.25,0.33,0,0.93,0.67,0.25,...,0.00,0.00,0.0,0.40,0.00,0.00,0.16,0.24,0.00,0.38
1,0.76,1,0.13,0.26,0.00,0.67,1,0.45,0.33,0.25,...,0.86,1.00,0.5,0.50,0.50,0.67,0.27,0.41,0.07,0.54
2,0.46,2,0.91,0.04,0.25,1.00,1,0.90,0.33,0.00,...,0.29,0.33,0.0,0.35,0.50,0.67,0.00,0.00,0.00,0.00
3,0.37,1,0.92,0.07,0.75,1.00,0,0.38,0.67,0.00,...,0.00,0.67,0.0,0.40,0.50,0.67,0.22,0.41,0.21,0.00
4,0.34,1,0.65,0.04,0.25,1.00,1,0.71,0.67,0.00,...,0.14,0.67,0.0,0.40,0.33,0.33,0.19,0.41,0.21,0.46


## Application d'algorithmes de clustering basé densité :

In [148]:
# Calculer la distance de manhattan entre tout les points du dataset
# Pour économiser du temps d'execution, on ne calcule que la matrice superieure de la matrice de distance
def manhattan_distance(df):
    distances = np.zeros((len(df), len(df)))
    for i in range(len(df)):
        for j in range(i+1, len(df)):
            dist = np.sum(np.abs(df[i] - df[j]))
            
            distances[i][j] = dist
            distances[j][i] = dist

    return distances

# Retourner tous les points du dataset qui sont à une distance inférieure à eps de notre instance
def region_query(distances, i, eps):
    doc_indexes = np.where(distances[i] <= eps)[0] # On récupère les index des points qui sont à une distance inférieure à eps
    return [i for i in doc_indexes] # On retourne la liste des index des points

# Fonction qui permet de récursivement ajouter les points voisins à notre cluster
def expand_cluster(distances, document_cluster, clusters, C, neighbors, eps, min_pts):
    while len(neighbors)!=0: # Tant qu'il y a des voisins
        index_doc = neighbors.pop() # On récupère le premier voisin
        if document_cluster[index_doc] >=0 : continue # Si le point appartient déjà à un cluster et n'est pas du bruit, on passe au suivant
        document_cluster[index_doc] = C # On change la valeur -1 par le numéro du cluster pour indiquer que le point appartient à ce cluster
        clusters[C].append(index_doc) # On ajoute le point au cluster
        new_neighbors = region_query(distances,index_doc,eps) # On récupère les points voisins du point
        if len(new_neighbors)>= min_pts: neighbors.extend(new_neighbors) # Si le point a assez de voisins, on ajoute les voisins au cluster

# Fonction recréant l'algorithme DBSCAN
def dbscan(distances, eps, min_pts):
    # On itilisalise une liste a -1 pour chaque point du dataset (on ne connait pas encore son cluster)
    document_cluster = np.full(len(distances), (-1), dtype = np.int)
    clusters = [[]]
    noise = []
    C = 0
    for index_doc in range(len(distances)):
        if document_cluster[index_doc] != -1 : continue # Si le point appartient déjà à un cluster, on passe au suivant
        
        neighbors = region_query(distances,index_doc,eps) # On récupère les points voisins
        
        if len(neighbors) >= min_pts: # Si le point a assez de voisins, on crée un nouveau cluster
            document_cluster[index_doc] = C # On change la valeur -1 par le numéro du cluster pour indiquer que le point appartient à ce cluster
            clusters[C].append(index_doc) # On ajoute le point au cluster
            expand_cluster(distances, document_cluster, clusters, C, neighbors, eps, min_pts) # On ajoute les voisins du point au cluster
            C += 1 # On passe au cluster suivant
            clusters.append([])  # On initialise le nouveau cluster
        else : # Si le point n'a pas assez de voisins, on le considère comme du bruit
            document_cluster[index_doc] = -2 # On change la valeur -1 par -2 pour indiquer que le point est du bruit
            noise.append(index_doc) # On ajoute le point au bruit

            
    del clusters[C] # On supprime le dernier cluster qui est vide
    
    return clusters, noise

In [149]:
# On calcule la matrice de distance au préalable pour économiser du temps d'execution
distances = manhattan_distance(data.to_numpy())
distances

array([[ 0.  , 10.46,  6.92, ...,  8.63,  7.47,  8.66],
       [10.46,  0.  ,  8.96, ...,  5.81,  6.95,  4.32],
       [ 6.92,  8.96,  0.  , ...,  7.21,  6.51,  6.9 ],
       ...,
       [ 8.63,  5.81,  7.21, ...,  0.  ,  7.58,  7.05],
       [ 7.47,  6.95,  6.51, ...,  7.58,  0.  ,  8.15],
       [ 8.66,  4.32,  6.9 , ...,  7.05,  8.15,  0.  ]])

In [150]:
# On teste l'algorithme DBSCAN avec différentes valeurs de eps et min_pts
min_pts = [2, 5, 7, 10, 15, 20, 25, 50, 100, 150, 200, 250, 300, 350, 400, 450, 500, 750, 1000]
results = []
for i in range(1, 16):
    for j in range(len(min_pts)):
        len_clusters = []
        clusters, noise = dbscan(distances, i, min_pts[j])
        for k in range(len(clusters)):
            len_clusters.append(len(clusters[k]))
        results.append([i, min_pts[j], len(clusters), len_clusters, len(noise)])
results = pd.DataFrame(results, columns = ['eps', 'min_pts', 'clusters', 'len_clusters', 'noise'])
results

,eps,min_pts,clusters,len_clusters,noise
0,1,2,0,[],1317
1,1,5,0,[],1317
2,1,7,0,[],1317
3,1,10,0,[],1317
4,1,15,0,[],1317
...,...,...,...,...,...
280,15,400,1,[1317],0
281,15,450,1,[1317],0
282,15,500,1,[1317],0
283,15,750,1,[1317],0


In [151]:
import random
nb_clusrers = results['clusters'].unique()
print(nb_clusrers)
results.loc[results['clusters'] == random.choice(nb_clusrers)]

[ 0 14 68  9  2  1 13]


,eps,min_pts,clusters,len_clusters,noise
39,3,5,9,"[256, 5, 4, 8, 3, 6, 4, 3, 6]",1042


## Application d'algorithmes de clustering basé hiérarchie

In [152]:
# La méthode Lien maximum est utilisée pour retouner a valeur maximale de toutes les
# distances par paires entre les éléments du cluster C1 et les éléments du cluster C2.
# 𝑑𝑖𝑠𝑡(𝐶1, 𝐶2) = 𝑀𝑎𝑥 ( 𝑑𝑖𝑠𝑡(𝑒1, 𝑒2) , 𝑒1 ∈ 𝐶1 𝑒𝑡 𝑒2 ∈ 𝐶2 ).
def complete_linkage(c1, c2):
    distances = np.zeros(len(c1) * len(c2))
    l = 0
    for i in range(len(c1)):
        for j in range(len(c2)):
            distances[l] = np.sum(np.abs(c1[i] - c2[j]))
            l += 1

    return np.amax(distances) # On retourne la valeur maximale de toutes les distances par paires entre les éléments du cluster C1 et les éléments du cluster C2

# La méthode Liaison minimale utilisée pour retouner a valeur minimale de toutes les
# distances par paires entre les éléments du cluster C1 et les éléments du cluster C2.
# 𝑑𝑖𝑠𝑡(𝐶1, 𝐶2) = 𝑀𝑖𝑛 ( 𝑑𝑖𝑠𝑡(𝑒1, 𝑒2) , 𝑒1 ∈ 𝐶1 𝑒𝑡 𝑒2 ∈ 𝐶2 ).
def single_linkage(c1, c2):
    distances = np.zeros(len(c1) * len(c2))
    l = 0
    for i in range(len(c1)):
        for j in range(len(c2)):
            distances[l] = np.sum(np.abs(c1[i] - c2[j]))
            l += 1

    return np.amin(distances) # On retourne la valeur minimale de toutes les distances par paires entre les éléments du cluster C1 et les éléments du cluster C2

# La méthode Liaison moyenne utilisée pour retouner a valeur moyenne de toutes les
# distances par paires entre les éléments du cluster C1 et les éléments du cluster C2.
# 𝑑𝑖𝑠𝑡(𝐶1, 𝐶2) = Σ(𝑒1 ∈ 𝐶1) Σ(𝑒2 ∈ 𝐶2 ) d𝑖𝑠𝑡(𝑒1, 𝑒2) / (𝑛1 * 𝑛2) où 𝑛1 et 𝑛2 sont les tailles des clusters C1 et C2.
def mean_linkage(c1, c2):
    distances = np.zeros(len(c1) * len(c2))
    l = 0
    for i in range(len(c1)):
        for j in range(len(c2)):
            distances[l] = np.sum(np.abs(c1[i] - c2[j]))
            l += 1

    return np.mean(distances) # On retourne la valeur moyenne de toutes les distances par paires entre les éléments du cluster C1 et les éléments du cluster C2

# La méthode Liaison centroid utilisée pour retouner la distance entre les centroides des deux clusters.
def centroid_linkage(c1, c2):
    centroid_c1 = np.mean(c1, axis=0)
    centroid_c2 = np.mean(c2, axis=0)
    return np.sum(np.abs(centroid_c1 - centroid_c2)) # On retourne la distance entre les centroides des deux clusters


def agglomeration_select_function(linkage):
    if linkage == 'complete':
        return complete_linkage
    elif linkage == 'single':
        return single_linkage
    elif linkage == 'average':
        return average_linkage
    elif linkage == 'centroid':
        return centroid_linkage

# On utiliser cette pour récupérer les index des points du dataset qui sont dans les cluster
def get_instance_index(clusters, data):

    clusters_index = [] # Liste des index des points du dataset qui sont dans les clusters

    for i in range(len(clusters)):

        clusters_temp = [] # Liste temporaire des index des points du dataset qui sont dans le cluster i
        
        for j in range(len(clusters[i])):
            # On récupère l'index du point a partir du dataset
            clusters_temp.append(Counter(np.where(data == clusters[i][j])[0]).most_common(1)[0][0])
        
        clusters_index.append(np.array(list(set(clusters_temp)))) # On ajoute les index du cluster i à la liste des clusters
    
    return np.array(clusters_index)

In [153]:
# Fonction recréant l'algorithme de clustering hiérarchique
def agglomerative_clustering(data, nb_clusters, linkage_method):

    # On initialise les clusters avec les points du dataset (chaque point est un cluster)
    clusters = np.asarray(list(map(lambda el:np.asarray([el]), data.to_numpy())))

    # On défini la fonction de calcul de distance en fonction du type de linkage
    linkage = agglomeration_select_function(linkage_method)

    while len(clusters) != nb_clusters: # Tant qu'on a pas le nombre de clusters voulu
        # On calcule la matrice de distance entre les clusters
        distances = np.zeros((len(clusters), len(clusters)))
        
        for i in range(len(clusters)):
            for j in range(i, len(clusters)):
                if i == j: # On met la distance entre un cluster et lui même à 100000 pour qu'il ne soit pas choisi
                    distances[i][j] = 100000

                else : # On calcule la distance entre le cluster i et le cluster j
                    dist = linkage(clusters[i], clusters[j]) 
                    # On met la distance entre le cluster i et le cluster j et la distance entre le cluster j et le cluster i à la même valeur
                    distances[i][j] = dist
                    distances[j][i] = dist

        temp_dist = distances.copy() # On copie la matrice de distance pour ne pas la modifier
        temps_clusters = clusters.copy() # On copie les clusters pour ne pas les modifier
        new_clusters = [] # Liste des nouveaux clusters

        while len(temp_dist) != 0: # Tant qu'il reste des clusters
            index = np.argmin(temp_dist, axis=1)[0] # On récupère l'index du cluster le plus proche du cluster 0
            new_clusters.append(np.concatenate((temps_clusters[0], temps_clusters[index]))) # On crée le nouveau cluster en concaténant les deux clusters les plus proches

            temps_clusters = np.delete(temps_clusters, 0,  axis= 0) # On supprime le cluster 0
            temp_dist = np.delete(temp_dist , 0,  axis= 0) # On supprime la liste des distances du cluster 0
            temp_dist = np.delete(temp_dist , 0,  axis= 1) # On supprime la distance entre le cluster 0 et les autres clusters

            if index != 0: # Si le cluster le plus proche du cluster 0 n'est pas le cluster 0
                temps_clusters = np.delete(temps_clusters, index - 1,  axis= 0) # On supprime le cluster le plus proche du cluster 0
                temp_dist = np.delete(temp_dist , index - 1,  axis= 0) # On supprime la liste des distances du cluster le plus proche du cluster 0
                temp_dist = np.delete(temp_dist , index - 1,  axis= 1) # On supprime la distance entre le cluster le plus proche du cluster 0 et les autres clusters
        
        clusters = new_clusters # On met à jour les clusters
    
    return get_instance_index(clusters, data) # On retourne les index des points du dataset qui sont dans les clusters

In [154]:
# On teste notre algorithmes
clusters = agglomerative_clustering(data, 2, 'complete')

In [155]:
# On vérifie la taille des clusters obtenus
len(clusters[0]), len(clusters[1])

(894, 423)

In [156]:
# On vérifie les classes des instances des clusters
df_test = pd.read_excel("data.xlsx")
df_test = df_test.drop(['Unnamed: 0'], axis = 1)
Counter(df_test.iloc[clusters[0]]["Attrition"]), Counter(df_test.iloc[clusters[1]]["Attrition"])

(Counter({1: 141, 0: 753}), Counter({0: 354, 1: 69}))